In [2]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 11.7 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


In [5]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
print('Execution Role: {}'.format(role))

bucket_name='octank-smartwatch-data'
s3_prefix='sagemaker-train'

data_key = 'subject101.csv'
data_location = 's3://{}/{}/{}'.format(bucket_name, s3_prefix, data_key)
print('S3 Data Location: {}'.format(data_location))

Execution Role: arn:aws:iam::572539092864:role/service-role/AmazonSageMaker-ExecutionRole-20200407T174741
S3 Data Location: s3://octank-smartwatch-data/sagemaker-train/subject101.csv


In [17]:
data = pd.read_csv(data_location, header=None)

In [24]:
# Apply column names
data.columns = ['timestamp', 'activity_id', 'heart_rate', 'imu_wrist_temp', 
                'imu_wrist_accel16_x', 'imu_wrist_accel16_y', 'imu_wrist_accel16_z', 
                'imu_wrist_accel6_x', 'imu_wrist_accel6_y', 'imu_wrist_accel6_z', 
                'imu_wrist_gyro_x', 'imu_wrist_gyro_y', 'imu_wrist_gyro_z', 
                'imu_wrist_magnet_x', 'imu_wrist_magnet_y', 'imu_wrist_magnet_z', 
                'imu_wrist_orient1', 'imu_wrist_orient2', 'imu_wrist_orient3', 'imu_wrist_orient4', 
                'imu_chest_temp', 'imu_chest_accel16_x', 'imu_chest_accel16_y', 'imu_chest_accel16_z', 
                'imu_chest_accel6_x', 'imu_chest_accel6_y', 'imu_chest_accel6_z', 
                'imu_chest_gyro_x', 'imu_chest_gyro_y', 'imu_chest_gyro_z', 
                'imu_chest_magnet_x', 'imu_chest_magnet_y', 'imu_chest_magnet_z', 
                'imu_chest_orient1', 'imu_chest_orient2', 'imu_chest_orient3', 'imu_chest_orient4', 
                'imu_ankle_temp', 'imu_ankle_accel16_x', 'imu_ankle_accel16_y', 'imu_ankle_accel16_z', 
                'imu_ankle_accel6_x', 'imu_ankle_accel6_y', 'imu_ankle_accel6_z', 
                'imu_ankle_gyro_x', 'imu_ankle_gyro_y', 'imu_ankle_gyro_z', 
                'imu_ankle_magnet_x', 'imu_ankle_magnet_y', 'imu_ankle_magnet_z', 
                'imu_ankle_orient1', 'imu_ankle_orient2', 'imu_ankle_orient3', 'imu_ankle_orient4']

data.columns

In [33]:
data.shape

(376417, 54)

In [34]:
data.head(5)

,timestamp,activity_id,heart_rate,imu_wrist_temp,imu_wrist_accel16_x,imu_wrist_accel16_y,imu_wrist_accel16_z,imu_wrist_accel6_x,imu_wrist_accel6_y,imu_wrist_accel6_z,...,imu_ankle_gyro_x,imu_ankle_gyro_y,imu_ankle_gyro_z,imu_ankle_magnet_x,imu_ankle_magnet_y,imu_ankle_magnet_z,imu_ankle_orient1,imu_ankle_orient2,imu_ankle_orient3,imu_ankle_orient4
0,8.38,0,104.0,30.0,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.008300,0.009250,-0.017580,-61.1888,-38.9599,-58.1438,1.0,0.0,0.0,0.0
1,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0
2,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0
3,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0
4,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0


In [35]:
data['timestamp'].head(5)

0    8.38
1    8.39
2    8.40
3    8.41
4    8.42
Name: timestamp, dtype: float64

In [36]:
data['activity_id'].head(5)

0    0
1    0
2    0
3    0
4    0
Name: activity_id, dtype: int64

In [37]:
data['heart_rate'].head(5)

0    104.0
1      NaN
2      NaN
3      NaN
4      NaN
Name: heart_rate, dtype: float64

In [38]:
data['imu_wrist_temp'].head(5)

0    30.0
1    30.0
2    30.0
3    30.0
4    30.0
Name: imu_wrist_temp, dtype: float64

In [9]:
activity_ids = ['unassigned', 'lying', 'sitting', 'standing ', 'walking', 'running', 'cycling', 'nordic_walking ', 
                'missing_8', 'watching_tv ', 'computer_work ', 'car_driving ', 'ascending_stairs ', 'descending_stairs ', 
                'missing_14', 'missing_15', 'vacuuming ', 'ironing', 'folding_laundry', 'house_cleaning', 'playing soccer', 
                'missing_21', 'missing_22', 'missing_23', 'rope jumping']